In [35]:
%load_ext autoreload
%autoreload 2

from pysat.formula import CNF
from compiler.entrypoint import Max3satQaoaCompiler

formula = CNF(from_file="./benchmarks/uuf100-01.cnf")

print(formula)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
CNF(from_string='c This Formular is generated by mcnf\nc\nc    horn? no\nc    forced? no\nc    mixed sat? no\nc    clause length = 3\nc\np cnf 100 430\n-46 90 77 0\n-100 -61 -2 0\n5 -95 79 0\n-34 -56 -23 0\n80 66 70 0\n-82 54 -34 0\n-77 28 33 0\n36 -17 -7 0\n-40 -90 -25 0\n55 -59 -46 0\n68 56 8 0\n74 -91 -62 0\n18 5 91 0\n94 58 37 0\n-98 74 -43 0\n66 4 67 0\n-9 -100 -46 0\n-82 64 -89 0\n-57 -48 -100 0\n51 -37 40 0\n-14 89 -40 0\n-20 -15 77 0\n30 -74 14 0\n-100 30 71 0\n97 57 -70 0\n73 -18 43 0\n-89 -18 -65 0\n15 13 30 0\n52 31 -22 0\n-26 -93 -25 0\n35 14 -22 0\n35 -29 100 0\n-24 21 -42 0\n85 -53 -15 0\n40 -81 -93 0\n-37 92 -93 0\n18 31 -98 0\n-32 66 9 0\n-66 4 -83 0\n49 -57 52 0\n13 -95 21 0\n-68 95 -55 0\n-54 44 -62 0\n-93 -82 85 0\n-46 -10 58 0\n27 -64 -12 0\n71 37 -85 0\n-21 56 -40 0\n63 -10 -37 0\n48 30 -8 0\n23 -55 34 0\n-72 28 7 0\n-17 84 -42 0\n61 -11 34 0\n-55 -26 -38 0\n83 7 19 0\n39 -80 -3

In [36]:
from nac.config import FPQAConfig

config = {
    "CCZ_GATE_FIDELITY": 0.99
}
fpqa_config = FPQAConfig(config)
print(fpqa_config.CCZ_GATE_FIDELITY)

compiler = Max3satQaoaCompiler(formula, fpqa_config)
program = compiler.compile_single_layer()
fidelity, duration = program.avg_fidelity(), program.duration()
print("Number of Instructions: ", len(program.instructions), ", Fidelity: ", fidelity, ", Duration ", duration)
print(program.count_ops())

0.99
Number of Instructions:  5335 , Fidelity:  1.767538927419421e-09 , Duration  62396.69439815551
{'u3': 3586, 'cz': 1220, 'ccz': 620}


In [32]:
from utils.hamiltonians import Max3satHamiltonian
from utils.qaoa import QAOA
from qiskit import transpile

basis_gates = ["u3", "id", "cz"]
hamiltonian = Max3satHamiltonian(formula=formula)
qaoa = QAOA(hamiltonian)
circuit, cost_params, mixer_params = qaoa.naive_qaoa_circuit(1)
bound_circuit = circuit.assign_parameters({cost_params: [0.2512], mixer_params: [0.1235]})
transpiled_circuit = transpile(bound_circuit, basis_gates=basis_gates, optimization_level=3)
print(bound_circuit.count_ops())
ops = transpiled_circuit.count_ops()
print("Fidelity: ", (0.999 ** ops["u3"]) * (0.995 ** ops["cz"]))
print(ops)


OrderedDict([('cx', 10200), ('rz', 4250), ('h', 250), ('rx', 250)])
Fidelity:  1.3127839693023915e-22
OrderedDict([('u3', 9658), ('cz', 8124)])


In [19]:
program.write("test.wqasm")